# Projeto 1 - Ciência dos Dados

Nome: Alexandre Cury

Nome: Celso Diniz

___
# Obtenção dos tweets

## Preparando o ambiente no jupyter:

In [8]:
#%%capture

#Instalando o tweepy
#!pip install tweepy

In [1]:
#import tweepy
#import math
#import os.path
#import pandas as pd
#import json
#from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[@ceodiniz]***

In [20]:
'''
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @ceodiniz

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])
''' 

"\n#Dados de autenticação do twitter:\n\n#Coloque aqui o identificador da conta no twitter: @ceodiniz\n\n#leitura do arquivo no formato JSON\nwith open('auth.pass') as fp:    \n    data = json.load(fp)\n\n#Configurando a biblioteca. Não modificar\nauth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])\nauth.set_access_token(data['access_token'], data['access_token_secret'])\n"

___
## Etapas para construção da base de dados:

### Escolha de um produto e coleta das mensagens


In [21]:

'''
#Produto escolhido:
produto = 'Playstation 5'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

''' 

"\n#Produto escolhido:\nproduto = 'Playstation 5'\n\n#Quantidade mínima de mensagens capturadas:\nn = 500\n#Quantidade mínima de mensagens para a base de treinamento:\nt = 300\n\n#Filtro de língua, escolha uma na tabela ISO 639-1.\nlang = 'pt'\n\n"

Capturando os dados do twitter:

In [23]:
'''
#Cria um objeto para a captura

api = tweepy.API(auth, wait_on_rate_limit=True) # respeita o limite e espera o periodo sem da erro ()
 
#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
msgs = list(tweepy.Cursor(api.search, q=produto + " -filter:retweets", lang=lang, tweet_mode="extended",count=n).items(n))
 
#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)
len(msgs)
msgs=list(set([msg.full_text.lower() for msg in msgs]))
print(len(msgs))
'''


'\n#Cria um objeto para a captura\n\napi = tweepy.API(auth, wait_on_rate_limit=True) # respeita o limite e espera o periodo sem da erro ()\n \n#Inicia a captura, para mais detalhes: ver a documentação do tweepy\ni = 1\nmsgs = []\nmsgs = list(tweepy.Cursor(api.search, q=produto + " -filter:retweets", lang=lang, tweet_mode="extended",count=n).items(n))\n \n#Embaralhando as mensagens para reduzir um possível viés\nshuffle(msgs)\nlen(msgs)\nmsgs=list(set([msg.full_text.lower() for msg in msgs]))\nprint(len(msgs))\n'

In [24]:
#msgs=list(set(list(msgs)))

Salvando os dados em uma planilha Excel:

In [25]:

'''
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()
'''

"\n#Verifica se o arquivo não existe para não substituir um conjunto pronto\nif not os.path.isfile('./{0}.xlsx'.format(produto)):\n    \n    #Abre o arquivo para escrita\n    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))\n\n    #divide o conjunto de mensagens em duas planilhas\n    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})\n    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)\n\n    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})\n    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)\n\n    #fecha o arquivo\n    writer.save()\n"

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

**Importante: Caso classifique um percentual pequeno de tweets relevantes ou de não relevantes, deve voltar a este notebook e coletar mais tweets diferentes do produto escolhido pelo grupo.**

In [26]:
import tweepy
import math
import os.path
import pandas as pd
#import json
from random import shuffle

In [27]:
data_class = pd.read_excel("PS5_classificado.xlsx")

In [28]:
data_class.head()

,Treinamento,Classificado
0,fortnite no playstation 5 no dia do lançamento...,0
1,o anúncio ocorre um dia após a sony brasil ter...,1
2,o playstation 5 é bonito? é! quero um? sim! te...,1
3,@playstation_br vou ao paraguai e trazer para ...,0
4,@stefano_poke nao viaja pelo amor de deus....\...,0


In [29]:
a = data_class["Classificado"].value_counts(True)
a

0    0.516667
1    0.483333
Name: Classificado, dtype: float64

In [30]:
relevante = data_class[data_class.Classificado == 1]
len(relevante)

145

In [31]:
irrelevante = data_class[data_class.Classificado == 0]
len(irrelevante)

155

## Categorizar itens por relevante e irrelevante

In [34]:
data_class["Classificado"] = data_class["Classificado"].astype('category')
data_class["Classificado"].cat.categories = ['relevante', 'irrelevante']

In [35]:
relevante = data_class["Classificado"] == 'relevante'
dados1 = data_class.loc[relevante, :]
dados1.Classificado.value_counts()[0]

155

In [36]:
irrelevante = data_class["Classificado"] == 'irrelevante'
dados2 = data_class.loc[irrelevante, :]
dados2.Classificado.value_counts()[1]

145

## Tabelas de frequencia Tweets Relevantes